#Data analysis


### pre-coding

Import libraries

In [1]:
from IPython.display import clear_output
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from datetime import datetime, timedelta
import os 
from google.colab import drive

Install missing libraries

In [3]:
!pip install xarray
!pip install netcdf4
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy
!pip install haversine
clear_output()

Import the recently installed libraries

In [4]:
import cartopy.crs as ccrs
from haversine import haversine, Unit
import xarray as xr

Connect to Google Drive to acces data files and set working directory

In [5]:
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Venture air challenge/Datasets")

Mounted at /content/drive


### Weekly analysis per region

Datasets, variables of interest and days of the week

In [6]:
datasets=["PAKILA_20x20m_7d.nc4","JATKASAARI_20x20m_7d.nc4","HELSINKI_100x100m_7d.nc4","DOWNTOWN_20x20m_7d.nc4"]
variables=["fmi_no2","fmi_no","fmi_pm10p0","fmi_pm2p5","fmi_so2","fmi_rel_humid","fmi_temp_2m","fmi_windspeed_10m","megasense_pm2p5","megasense_co","megasense_pm10p0","megasense_aqi","megasense_o3","megasense_no2"]
days = ["Monday","Tusday","Wednesday","Thursday","Friday","Saturday","Sunday"]

We plotted the data for all the variables related to air quality for the different regions for one week. We average the data each hour and used histograms to represent the results.

In [ ]:
for dataset in datasets: 
  xds = xr.open_dataset("./"+dataset, decode_times=False) # Open dataset
  
  for var in variables:
    data=xds[var]
    hourlyData=[]

    for hour in range(168):
      fmi_PM10=data[(hour*12):((hour*12)+12)] # Get data for one hour
      fmi_PM10_t = fmi_PM10.mean("time") # Average over time
      nansFreeData=fmi_PM10_t.values[np.logical_not(np.isnan(fmi_PM10_t.values))] # Discard NaNs
      hourlyData.append(np.mean(nansFreeData))
      
    # Set plot
    fig = plt.figure(figsize=(20,4))

    # Plot weekly data
    for i in range(7):
      hours = np.linspace(i*24,(i*24+23),24)
      plt.bar(hours,hourlyData[(i*24):(i*24+24)],width=0.5,label=days[i])
    mpl.rc('xtick', labelsize=13) 
    mpl.rc('ytick', labelsize=13) 
    plt.ylabel(var+r' ($\mu g/m^3$)',fontsize=16)
    plt.xlabel("Time (h)",fontsize=15)
    plt.legend(fontsize=16,loc=1)
    plt.savefig("graficas/"+var+data.split("_")[0]+".png",dpi=800)
    plt.close()

Then we also used the 30 day data and divided it by weeks to perform a similar analisys to the one done for the 7 day data

In [ ]:
xds = xr.open_dataset("./HELSINKI_100x100m_3mo.nc4"+dataset, decode_times=False) # Open dataset

for var in variables:
  xdsVar=xds[var]
  for week in range(13):
    data=xdsVar[(2016*week):(2016*week+2016)]
    hourlyData=[]
    for hour in range(168):
      fmi_PM10=data[(hour*12):((hour*12)+12)] # Get data for one hour
      fmi_PM10_t = fmi_PM10.mean("time") # Average over time
      nansFreeData=fmi_PM10_t.values[np.logical_not(np.isnan(fmi_PM10_t.values))] # Discard NaNs
      hourlyData.append(np.mean(nansFreeData))
      
    # Set plot
    fig = plt.figure(figsize=(20,4))

    # Plot weekly data
    for i in range(7):
      hours = np.linspace(i*24,(i*24+23),24)
      plt.bar(hours,hourlyData[(i*24):(i*24+24)],width=0.5,label=days[i])
    mpl.rc('xtick', labelsize=13) 
    mpl.rc('ytick', labelsize=13) 
    plt.ylabel(var+r' ($\mu g/m^3$)',fontsize=16)
    plt.xlabel("Time (h)",fontsize=15)
    plt.legend(fontsize=16,loc=1)
    plt.title("Week "+str(w+1))
    plt.savefig("graficas3m/"+var+"_week"+str(w)+"_"+data.split("_")[0]+".png",dpi=800)
    plt.close()

###Data in context

We also ploted data for different variables in a map, for this we average over time. In this case PM 10 in Helsinki

In [ ]:
data="HELSINKI_100x100m_7d.nc4"
xds = xr.open_dataset("./"+data, decode_times=False) 
fmi_PM10=xds.megasense_pm10p0
fmi_PM10_t = fmi_PM10.mean("time")

fig, ax = plt.subplots(figsize=(8,5))
diagram=fmi_PM10_t.plot()
ax.set_title("PM 10 concentration in Helsinki",fontsize=15)
ax.set_ylabel("Latitude",fontsize=15)
ax.set_xlabel("Longitude",fontsize=15)
plt.gca().collections[-1].colorbar.remove()
cbar = fig.colorbar(diagram)
diagram.set_clim(vmin=0, vmax=70)
mpl.rc('xtick', labelsize=13) 
mpl.rc('ytick', labelsize=13)
plt.axis('scaled')
plt.savefig("coordenadas/pm25.png",dpi=800)
plt.close()

Afterwards we filtered data based on certain criteria, in this case PM 10 concentration levels of 20 micrograms per cubic meter

In [ ]:
badpm10=fmi_PM10_t.where(fmi_PM10_t>30)
fig, ax = plt.subplots(figsize=(8,5))
diagram=badpm10.plot()
ax.set_title("PM 10 concentration in Helsinki",fontsize=15)
ax.set_ylabel("Latitude",fontsize=15)
ax.set_xlabel("Longitude",fontsize=15)
plt.gca().collections[-1].colorbar.remove()
cbar = fig.colorbar(diagram)
diagram.set_clim(vmin=0, vmax=70)
mpl.rc('xtick', labelsize=13) 
mpl.rc('ytick', labelsize=13)
plt.axis('scaled')
plt.savefig("coordenadas/pm25Bad.png",dpi=800)
plt.close()

###Bad spots and tower locations

We identified bad spots in the cities using Megasense PM 2.5 particle concentration above 30 micrograms per cubic meter and we stored the coordinates, alongside we found the best locations for the Purify towers

In [ ]:
data="HELSINKI_100x100m_7d.nc4"
xds = xr.open_dataset("./"+data, decode_times=False) 
xds=xds.mean("time")
var="megasense_pm2p5"
thr=30
allLocations=[]
badLocations=[]
towerLocations=[]

for i in xds.lat.values:
    for j in xds.lon.values:
        cell = xds.loc[dict(lat=i, lon=j)]
        df = cell[var].to_dataframe()
        if pd.notna(df[var].values):
          allLocations.append((df[var].values)[0])
          if df[var].values>thr:
            badLocations.append([df[var].values[0],i,j])
            n = True
            for f in towerLocations:
              if haversine((i,j), (f[1],f[2]))<0.5:
                n = False
                break
            if n:
              towerLocations.append([df[var].values[0],i,j])

And then we saved the locations in txt files

In [ ]:
allLocations=np.array(allLocations)
f = open("allLocations.txt", "w")
f.write("PM25,Lat,Lot\n")
for i in range(np.size(allLocations[:,0])):
  f.write("%3.8f,%3.8f,%3.8f\n"%(allLocations[i,0],allLocations[i,1],allLocations[i,2]))
f.close()